In [1]:
import pandas as pd
import glob
from huggingface_hub import hf_hub_download

In [2]:
# gt と result の一致性を確認
result_path="/home/ec2-user/FinanceRAG/results/"

# result_path の下で 2024-で始まるディレクトリを列挙
import os
dirs = os.listdir(result_path)
dirs=[d for d in dirs if d.startswith("2024-11-08-13")]
trainig_data=pd.DataFrame()

for d in dirs:
    print(d)
    for t in ["FinQABench", "FinDER", "FinanceBench", "TATQA", "FinQA", "ConvFinQA", "MultiHiertt"]:
    #for t in ["MultiHiertt"]:
        try:
            # read FinDER_qrels.tsv
            gt=pd.read_csv('../'+t+'_qrels.tsv', sep='\t')

            gt['query_id']=gt['query_id'].astype(str)

            p = glob.glob(result_path+"/"+d+"/"+t+"/*/results.csv")
            result=pd.read_csv(p[0])
            result['query_id']=result['query_id'].astype(str)

            #resultとgtを結合し、corpus_idが一致していないものをnegativeエントリ、一致しているものをpositiveエントリとしてtrainig_dataに追加

            result_gt=pd.merge(result, gt, on='query_id', suffixes=('_result', '_gt'), how='inner')
            result_gt['flag']=0
            true_data=result_gt[result_gt['corpus_id_result']==result_gt['corpus_id_gt']]
            true_data['flag']=1
            false_data=result_gt[result_gt['corpus_id_result']!=result_gt['corpus_id_gt']]
            trainig_data=pd.concat([trainig_data, false_data])
            trainig_data=pd.concat([trainig_data, true_data])


            num_correct=result_gt[result_gt['corpus_id_result']==result_gt['corpus_id_gt']].shape
            print("ratio_correct: ",t, num_correct[0]/gt.shape[0])
        except:
            print("no result for ", t)


2024-11-08-13
ratio_correct:  FinQABench 0.9333333333333333
ratio_correct:  FinDER 0.44660194174757284
ratio_correct:  FinanceBench 0.8305084745762712
ratio_correct:  TATQA 0.5120481927710844
ratio_correct:  FinQA 0.47674418604651164
ratio_correct:  ConvFinQA 0.5238095238095238
ratio_correct:  MultiHiertt 0.08796992481203007


/tmp/ipykernel_24377/1067254219.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_data['flag']=1
/tmp/ipykernel_24377/1067254219.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  true_data['flag']=1
/tmp/ipykernel_24377/1067254219.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#return

In [3]:
#trainig_data に quertとcorpusを入れる
# query と coupasを読み込み結合
data_all=pd.DataFrame()
query_all=pd.DataFrame()
truth_name_list=["finqabench", "finder", "financebench", "tatqa", "finqa", "convfinqa", "multiheirtt"]
for ind, t in enumerate(["FinQABench", "FinDER", "FinanceBench", "TATQA", "FinQA", "ConvFinQA", "MultiHiertt"]):
    print(t)
    data=pd.read_json('../'+truth_name_list[ind]+'_corpus.jsonl/corpus.jsonl', lines=True)
    query=pd.read_json('../'+truth_name_list[ind]+'_queries.jsonl/queries.jsonl', lines=True)
    data_all=pd.concat([data_all, data])
    query_all=pd.concat([query_all, query])

# トレーニングデータにqueryとcorpusを追加
trainig_data=pd.merge(trainig_data, data_all[["_id","title","text"]], left_on='corpus_id_result',right_on='_id', how='left')
trainig_data=pd.merge(trainig_data, query_all[["_id","title","text"]], left_on='query_id', right_on='_id', how='left')


FinQABench
FinDER
FinanceBench
TATQA
FinQA
ConvFinQA


MultiHiertt


In [4]:
from sentence_transformers import InputExample

positive_examples = []
negative_examples = []

for idx, row in trainig_data.iterrows():
    # Ensure the negative example is not the same as the positive one
    if row['flag'] == 1:
        positive_examples.append(
            InputExample(texts=[row['title_y']+row['text_y'], row['text_x']], label=1.0)
        )
    else:
        negative_examples.append(
            InputExample(texts=[row['title_y']+row['text_y'], row['text_x']], label=0.0)
        )

In [5]:
pip show huggingface_hub


Name: huggingface-hub
Version: 0.26.2
Summary: Client library to download and publish models, datasets and other repos on the huggingface.co hub
Home-page: https://github.com/huggingface/huggingface_hub
Author: Hugging Face, Inc.
Author-email: julien@huggingface.co
License: Apache
Location: /opt/conda/envs/pytorch/lib/python3.11/site-packages
Requires: filelock, fsspec, packaging, pyyaml, requests, tqdm, typing-extensions
Required-by: accelerate, datasets, sentence-transformers, tokenizers, transformers
Note: you may need to restart the kernel to use updated packages.


In [6]:
# Combine positive and negative examples
train_examples = positive_examples + negative_examples

In [7]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16)  # Adjust batch size as needed


In [8]:
from sentence_transformers import SentenceTransformer, losses

# Load a pre-trained model
model = SentenceTransformer('intfloat/e5-large-v2')


In [9]:
from transformers import Trainer

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        """
        Custom loss function that accepts additional arguments with **kwargs.
        """
        # Forward pass through the model
        outputs = model(**inputs)

        # Compute loss based on model output
        loss = outputs.loss if isinstance(outputs, dict) else outputs[0]

        if return_outputs:
            return loss, outputs
        return loss

In [10]:
train_loss = losses.CosineSimilarityLoss(model)


In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=1,  # Adjust epochs based on your dataset size and desired training time
    warmup_steps=100,  # Adjust based on dataset size
    evaluator=None,  # If no evaluator is required, set this to None
    output_path=None,  # Provide output_path if you want to save the model
    show_progress_bar=True,  # Whether to show a progress bar
)

Step,Training Loss


In [ ]:
# Save the fine-tuned model
model.save('fine_tuned_sentence_transformer_e5-large-v2')

Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]